In [1]:
# The data will be downloaded
from tensorflow.examples.tutorials.mnist import input_data
# Cached the data into MNIST_data folder.
mnist_data = input_data.read_data_sets('MNIST_data', one_hot=True)

import tensorflow as tf

print 'finish'

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
finish


In [2]:
# Defining placeholders for input data and targets
input_size = 784
no_classes = 10
batch_size = 100
total_batches = 200

# None indicates that it can be of any size
x_input = tf.placeholder(tf.float32, shape=[None, input_size])
y_input = tf.placeholder(tf.float32, shape=[None, no_classes])

print x_input
print y_input

print 'finish'

Tensor("Placeholder:0", shape=(?, 784), dtype=float32)
Tensor("Placeholder_1:0", shape=(?, 10), dtype=float32)
finish


Nesta seção, veremos como criar uma rede convolucional de multicamadas no TensorFlow e observar como uma rede mais profunda melhora a precisão da classificação. Vamos definir as camadas com as APIs das camadas do TensorFlow em vez de defini-las do zero. Os métodos de melhores práticas estão enraizados nesses métodos. A importação de bibliotecas, conjuntos de dados e espaços reservados pode ser seguida da seção anterior. Desta vez, usaremos o TensorBoard para visualizar o processo de treinamento. Para visualizar as estatísticas das variáveis, os valores das estatísticas variáveis devem ser adicionados ao arquivo tf.summary

In [3]:
def add_variable_summary(tf_variable, summary_name):
    with tf.name_scope(summary_name + '_summary'):
        mean = tf.reduce_mean(tf_variable)
        tf.summary.scalar('Mean', mean)
        
        with tf.name_scope('standard_deviation'):
            standard_deviation = tf.sqrt(tf.reduce_mean(
               tf.square(tf_variable - mean)))
            
        tf.summary.scalar('StandardDeviation', standard_deviation)
        tf.summary.scalar('Maximum', tf.reduce_max(tf_variable))
        tf.summary.scalar('Minimum', tf.reduce_min(tf_variable))
        tf.summary.histogram('Histogram', tf_variable)

Ao contrário do modelo anterior, vamos redimensionar os dados MNIST em um quadrado e usá-lo como uma imagem bidimensional. O seguinte é o comando para remodelar a imagem em 28 pixels de imagem por 28 pixels de imagem:

In [4]:
x_input_reshape = tf.reshape(x_input, [-1, 28, 28, 1], name='input_reshape')

A dimensão -1 denota que o tamanho do lote pode ser qualquer número. Observe que há um argumento chamado nome que será refletido no gráfico TensorBoard para facilitar a compreensão. Vamos definir uma camada de convolução 2D onde a entrada, os filtros, os kernels e as ativações são definidos. Este método pode ser chamado em qualquer lugar para exemplos adicionais e é útil quando a função de ativação precisa ter a ativação da Unidade Linear Retificada (ReLU). A camada de função de convolução é definida da seguinte forma:

In [5]:
def convolution_layer(input_layer, filters, kernel_size=[3, 3], activation=tf.nn.relu):
    layer = tf.layers.conv2d(
        inputs=input_layer,
        filters=filters,
        kernel_size=kernel_size,
        activation=activation,
    )
    add_variable_summary(layer, 'convolution')
    return layer

Existem parâmetros padrão para o kernel_size e a ativação. Os resumos são adicionados à camada dentro da função e a camada é retornada. Sempre que a função é chamada, input_layer deve ser passado como um parâmetro. Esta definição tornará nosso outro código simples e pequeno. De forma muito semelhante, vamos definir uma função para o pooling_layer da seguinte forma

In [6]:
def pooling_layer(input_layer, pool_size=[2, 2], strides=2):
    layer = tf.layers.max_pooling2d(
        inputs=input_layer,
        pool_size=pool_size,
        strides=strides
    )
    add_variable_summary(layer, 'pooling')
    return layer

Esta camada tem parâmetros padrão para pool_size e strides para [2, 2] e 2, respectivamente. Esses parâmetros geralmente funcionam bem, mas podem ser alterados quando necessário. Os resumos são adicionados para essa camada também. Em seguida, definiremos uma camada densa da seguinte forma: 

In [7]:
def dense_layer(input_layer, units, activation=tf.nn.relu):
    layer = tf.layers.dense(
        inputs=input_layer,
        units=units,
        activation=activation
    )
    add_variable_summary(layer, 'dense')
    return layer

A camada densa definida possui parâmetros padrão para ativação e os resumos de variáveis também são adicionados. O pooling_layer pega o mapa de recursos da camada de convolução e reduz a metade desse tamanho pulando, usando o tamanho do pool e os passos. Todas essas camadas estão conectadas como um gráfico e são apenas definidas. Nenhum dos valores é inicializado. Outra camada de convolução pode ser adicionada para transformar os recursos amostrados da primeira camada de convolução em melhores recursos. Após o agrupamento, podemos reformular as ativações de maneira linear para sermos alimentados por camadas densas:

In [8]:
convolution_layer_1 = convolution_layer(x_input_reshape, 64)
print convolution_layer_1

pooling_layer_1 = pooling_layer(convolution_layer_1)
print pooling_layer_1

convolution_layer_2 = convolution_layer(pooling_layer_1, 128)
print convolution_layer_2

pooling_layer_2 = pooling_layer(convolution_layer_2)
print pooling_layer_2

flattened_pool = tf.reshape(pooling_layer_2, [-1 , 5 * 5 * 128], name='flattened_pool')
print flattened_pool

dense_layer_bottleneck = dense_layer(flattened_pool, 1024)
print dense_layer_bottleneck

Tensor("conv2d/Relu:0", shape=(?, 26, 26, 64), dtype=float32)
Tensor("max_pooling2d/MaxPool:0", shape=(?, 13, 13, 64), dtype=float32)
Tensor("conv2d_2/Relu:0", shape=(?, 11, 11, 128), dtype=float32)
Tensor("max_pooling2d_2/MaxPool:0", shape=(?, 5, 5, 128), dtype=float32)
Tensor("flattened_pool:0", shape=(?, 3200), dtype=float32)
Tensor("dense/Relu:0", shape=(?, 1024), dtype=float32)


A única diferença entre as camadas de convolução é o tamanho do filtro. É importante que as dimensões mudem apropriadamente de camada para camada. A escolha dos parâmetros para kernel e stride é arbitrária e esses números são escolhidos pela experiência. Duas camadas de convolução são definidas, e isso pode ser seguido por uma camada totalmente conectada. Uma API de camada densa pode usar qualquer vetor de uma única dimensão e mapeá-la para qualquer número de unidades ocultas, como neste caso é 1024. A camada oculta é seguida pela ativação de ReLU para tornar isso uma computação não linear. Resumos variáveis são adicionados para essa camada também. Isto é seguido por uma camada de abandono com uma taxa de abandono. Manter isso alto impedirá que a rede aprenda. O modo de treinamento pode ser definido como Verdadeiro e Falso com base em quando usamos isso. Vamos definir isso como True (o padrão é False) para o treinamento. Teremos que mudar isso enquanto a precisão é calculada. Assim, um bool é mantido para isso, que será alimentado durante o treinamento:

In [9]:
dropout_bool = tf.placeholder(tf.bool)
dropout_layer = tf.layers.dropout(
    inputs=dense_layer_bottleneck,
    rate=0.4,
    training=dropout_bool
)

A camada de dropout é alimentada novamente para uma camada densa, que é chamada de logits. Logits é a camada final com ativações que levam ao número de classes. As ativações serão adicionadas a uma classe específica, que é a classe alvo, e podem ser obtidas para um máximo dessas 10 ativações:

In [10]:
logits = dense_layer(dropout_layer, no_classes)
print logits

Tensor("dense_2/Relu:0", shape=(?, 10), dtype=float32)


A saída de logits é muito semelhante ao modelo criado na seção anterior. Agora os logits podem ser passados através da camada softmax seguida pelo cálculo de entropia cruzada como antes. Aqui, adicionamos um nome de escopo para obter uma melhor visualização no TensorBoard da seguinte maneira

In [11]:
with tf.name_scope('loss'):
    softmax_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
        labels=y_input, logits=logits)
    loss_operation = tf.reduce_mean(softmax_cross_entropy, name='loss')
    tf.summary.scalar('loss', loss_operation)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



Esta função de perda pode ser otimizada com os métodos das APIs do tf.train. Aqui, vamos usar o Adamoptimiser. A taxa de aprendizado não precisa ser definida e funciona bem para a maioria dos casos:

In [12]:
with tf.name_scope('optimiser'):
    optimiser = tf.train.AdamOptimizer().minimize(loss_operation)

A precisão é calculada como antes, mas escopos de nome são adicionados para previsões corretas e cálculo de precisão:

In [13]:
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        predictions = tf.argmax(logits, 1)
        correct_predictions = tf.equal(predictions, tf.argmax(y_input, 1))
    with tf.name_scope('accuracy'):
        accuracy_operation = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
tf.summary.scalar('accuracy', accuracy_operation)

<tf.Tensor 'accuracy_1:0' shape=() dtype=string>

Um resumo escalar para precisão também é adicionado. O próximo passo é iniciar a sessão e inicializar as variáveis como na seção anterior. As linhas não são repetidas aqui. Os resumos devem ser mesclados, e os arquivos para redigir os resumos de treinamento e teste devem ser definidos:

In [14]:
session = tf.Session()
session.run(tf.global_variables_initializer())

merged_summary_operation = tf.summary.merge_all()
train_summary_writer = tf.summary.FileWriter('/tmp/train', session.graph)
test_summary_writer = tf.summary.FileWriter('/tmp/test')

Observe que o gráfico é escrito uma vez com o summary_writer. O treinamento é muito semelhante ao anterior, exceto que os cálculos de precisão durante o treinamento e os valores são adicionados aos resumos. Em seguida, os dados podem ser carregados em lotes e o treinamento pode ser iniciado:

In [15]:
test_images, test_labels = mnist_data.test.images, mnist_data.test.labels

for batch_no in range(total_batches):
    mnist_batch = mnist_data.train.next_batch(batch_size)
    train_images, train_labels = mnist_batch[0], mnist_batch[1]
    _, merged_summary = session.run([optimiser, merged_summary_operation],
                                    feed_dict={
        x_input: train_images,
        y_input: train_labels,
        dropout_bool: True
    })
    train_summary_writer.add_summary(merged_summary, batch_no)
    if batch_no % 10 == 0:
        merged_summary, _ = session.run([merged_summary_operation,
                                         accuracy_operation], feed_dict={
            x_input: test_images,
            y_input: test_labels,
            dropout_bool: False
        })
        test_summary_writer.add_summary(merged_summary, batch_no)

Para ver o resultado do treinamento deve-se usar o TensorBoard

Os resumos são retornados em cada iteração para dados de treinamento e são adicionados ao gravador. Para cada décima iteração, os resumos de teste são adicionados. Observe que o dropout é ativado apenas durante o treinamento e não durante o teste. Completamos a definição e os resumos da rede e isso pode ser executado. Para ver o processo de treinamento, podemos acessar o TensorBoard, conforme descrito no Capítulo 1, Introdução.